**Problem Statement:**

Create a machine learning algorithm that can predict if a customer will buy the audio book again or not.

**PLAN OF ACTION:**



*   Preprocess the data:

Balance the Dataset

Divide the dataset in training, validation, and test

save the data in a tensor friendly format(.npz format)
*   Create the machine learning algorithm



------------------------------------------------------

**Extract the data from the csv**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
from sklearn import preprocessing
raw_csv_data = np.loadtxt('/content/drive/MyDrive/Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

**Balance the Dataset**

In [5]:
num_of_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_targets_counter += 1
    if zero_targets_counter > num_of_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)

**Standardize the inputs**

In [6]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

**Shuffle the Data**

In [7]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

**Split the dataset into train, validation, and test**

In [10]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1768.0 3579 0.49399273540095
233.0 447 0.5212527964205816
236.0 448 0.5267857142857143


**Save the three Datasets in** **.npz **

In [12]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

**Create a Machine Learning Algorithm**

In [13]:
import tensorflow as tf

**Data**

In [15]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(float)
validation_targets = npz['targets'].astype(int)

npz = np.load("Audiobooks_data_test.npz")

test_inputs = npz['inputs'].astype(float)
test_targets = npz['targets'].astype(int)

**Model**

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                          tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                          tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                          tf.keras.layers.Dense(hidden_layer_size, activation = 'softmax')
                          ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2)

**Test the Model**

In [19]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7789 - loss: 0.3772 


In [21]:
print('\nTest loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.38. Test Accuracy: 78.12%
